In [1]:
import numpy as np
import mygrad as mg
from mygrad import Tensor
from pathlib import Path
import matplotlib.pyplot as plt
import base64
from IPython.display import display, HTML
%matplotlib notebook

In [2]:
_config_file = Path.home() / ".datasets"
def get_path(verbose=False):
    if _config_file.is_file():
        with _config_file.open("r") as f:
            header, path = f.readlines()
        path = Path(path)
    else:
        path = Path.home() / "datasets"
        path.mkdir(exist_ok=True)
    if verbose:
        print("`datasets module: datasets will be loaded from '{}'".format(path))
    return path

In [3]:
def load_mnist(fname="mnist.npz"):
    with np.load(fname) as data:
        out = tuple(data[str(key)] for key in ['x_train', 'y_train', 'x_test', 'y_test'])
    print("mnist loaded")
    return out

In [4]:
# mnist 데이터셋을 불러와서 학습용/테스트용 이미지/라벨 을 저장
x_train, y_train, x_test, y_test = load_mnist()

mnist loaded


In [5]:
print(type(x_train), x_train.shape, x_train.dtype)
print(type(y_train), y_train.shape, y_train.dtype)
print(type(x_test), x_test.shape, x_test.dtype)
print(type(y_test), y_test.shape, y_test.dtype)

<class 'numpy.ndarray'> (60000, 1, 28, 28) uint8
<class 'numpy.ndarray'> (60000,) uint8
<class 'numpy.ndarray'> (10000, 1, 28, 28) uint8
<class 'numpy.ndarray'> (10000,) uint8


In [7]:
img_id = 0 # 0 ~ 59999 사이의 정수 입력

fig, ax = plt.subplots()
ax.imshow(x_train[img_id, 0], cmap="gray")
ax.set_title(f"truth: {y_train[img_id]}")

<IPython.core.display.Javascript object>

In [8]:
# 여기에 코드 작성
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [ ]:
# 여기에 코드 작성
x_train /= 255.0
x_test /= 255.0

In [9]:
# 패딩을 지정해주는 코드
x_train = np.pad(x_train, ((0, 0), (0, 0), (2, 2), (2, 2)), mode="constant")
x_test = np.pad(x_test, ((0, 0), (0, 0), (2, 2), (2, 2)), mode="constant")

In [10]:
import mynn

In [11]:
from mynn.layers.conv import conv
from mynn.layers.dense import dense

from mygrad.nnet.initializers import glorot_uniform
from mygrad.nnet.activations import relu
from mygrad.nnet.layers import max_pool

class Model:
    ''' 간단한 CNN (Convolutional Neural Network) '''
    
    def __init__(self, num_input_channels, f1, f2, d1, num_classes):
        """
        클래스 생성 시 실행되어 계층과 가중치들을 초기화하는 역할.
        
        매개변수 (Parameters)
        ----------
        num_input_channels : int
            입력 데이터(이미지)의 색상 채널 수
            
        f1 : int
            첫번째 계층 _ 합성곱 계층의 필터 수
        
        f2 : int
            두번째 계층 _ 합성곱 계층의 필터 수

        d1 : int
            세번째 계층 _ 밀집층의 뉴런 수
        
        num_classes : int
            모델의 분류 항목 수
            네번째 계층 _ 밀집층의 뉴런 수
        """
        # 두 개의 합성곱 계층과 두 개의 밀집층을 각각 초기화
        # MyNN의 conv(), dense() 함수 사용
        # weight_initializer = glorot_uniform, gain = np.sqrt(2) 로 설정
        
        # 주의: 두 번째 계층의 계산 결과로 얻은 이미지 채널을 벡터화(vectorization, flatten)한 결과가
        # 세 번째 계층으로 들어가므로, 세 번째 계층의 밀집층의 input_size를 잘 계산해야 함
        
        # 여기에 코드 작성
        init_kwargs = {'gain': np.sqrt(2)}
        
        self.conv1 = conv(num_input_channels, f1, 
                          5, 5, 
                          weight_initializer=glorot_uniform, 
                          weight_kwargs=init_kwargs)
        
        self.conv2 = conv(f1, f2, 5, 5, weight_initializer=glorot_uniform, weight_kwargs=init_kwargs)
        
        self.dense1 = dense(f2 * 5 * 5, d1,   # 여기서 input, output 구하는 방법 중요함 - 데이터에 따라 달라짐 
                            weight_initializer=glorot_uniform, 
                            weight_kwargs=init_kwargs)
        
        self.dense2 = dense(d1, num_classes, weight_initializer=glorot_uniform, weight_kwargs=init_kwargs)


    def __call__(self, x):
        '''
        입력 데이터에 따른 모델의 순전파(Forward Propagation)를 수행
        
        매개변수 (Parameters)
        ----------
        x : numpy.ndarray, shape=(N, 1, 32, 32)
            입력 데이터. N장의 이미지
            
        반환 값 (Returns)
        -------
        mygrad.Tensor, shape=(N, num_classes)
            N장의 이미지에 대해 분류 항목별 예측값을 배열로 반환
            Softmax 통과 전이므로 확률이 아님에 주의
        '''
        # 모델의 구조를 잘 생각하며 모델의 순전파를 수행하는 코드 작성
        # 가중치를 포함하는 합성곱 계층 및 밀집층은 __init__에서 정의함
        # 활성함수 relu와 최대 풀링 함수 max_pool을 추가로 이용하여 작성하면 됨
        
        # 이미지 채널을 밀집층에 넣어주기 위한 벡터화 진행 시,
        # N개의 이미지에 대해 개별적으로 벡터화를 진행해야 함 (HINT > NumPy ndarray의 reshape() 메서드 사용) 
        
        # 여기에 코드 작성
        x = relu(self.conv1(x)) # ____ : activation function
        x = max_pool(x, (2, 2), 2)
        x = relu(self.conv2(x)) # conv layer 2
        x = max_pool(x, (2, 2), 2) # max pooling
        x = relu(self.dense1(x.reshape(x.shape[0], -1))) # flatten + 1st dense layer + relu function
        x = self.dense2(x) # 2nd dense layer
        return x

    @property
    def parameters(self):
        """
        모델 파라미터를 리스트 형태로 전부 가져올 수 있는 유용한 함수
        데코레이터 @property를 붙였기에 메서드가 아닌 속성처럼 사용해야 함.
        즉, model.parameters()가 아닌 model.parameters로 호출
        
        반환 값 (Returns)
        -------
        List[Tensor, ...]
            모델의 학습 가능한 모델 파라미터들을 모아놓은 리스트
        """
        # __init__에서 정의한 4개의 계층에 포함된 모든 가중치(모델 파라미터) 반환
        # 각 레이어의 파라미터를 self.conv.parameters와 같이 가져올 수 있음
        # for문 이용하기
        
        # 여기에 코드 작성
        params = []
        for layer in (self.conv1, self.conv2, self.dense1, self.dense2):
            params += list(layer.parameters) # 각 layer의 파라미터를 리스트로 담아서 가져온다. 
        return params

In [12]:
def accuracy(predictions, truth):
    """
    하나의 batch에 대한 모델의 예측값과 실제값을 비교하여, 정확도를 계산하는 함수
    
    매개변수 (Parameters)
    ----------
    predictions : Union[numpy.ndarray, mg.Tensor], shape=(M, D)
        D개의 분류 항목에 대해 예측한 각 확률을 batch의 데이터 M개에 대해 배열로 정리
        이 실습에서는 분류할 항목의 개수가 숫자 0~9, 총 10개이므로 D = 10
        
    truth : numpy.ndarray, shape=(M,)
        데이터 M개에 대응되는 정답 라벨들로 이루어진 배열
        각 라벨은 D개의 분류 항목 [0, D) 중 하나의 값
            
    반환 값 (Returns)
    -------
    float
        올바른 예측의 비율 (0 이상 1 이하의 실수값)
        해당 batch에 대한 모델의 분류 정확도
    """
    # 여기에 코드 작성
    return np.mean(np.argmax(predictions, axis = 1) == truth)

In [14]:
# SGD를 import하여
# Optimizer로 모델을 초기화

from mynn.optimizers.sgd import SGD

model = Model(f1=20, f2=10, d1=20, num_input_channels=1, num_classes=10) # 빈칸 채우기. 이번 실습은 몇개의 class를 분류해야 할까요?
optim = SGD(model.parameters, learning_rate=0.01, momentum=0.9, weight_decay=5e-04)

In [15]:
from noggin import create_plot

plotter, fig, ax = create_plot(["loss", "accuracy"])

<IPython.core.display.Javascript object>

In [19]:
from mygrad.nnet.losses import softmax_crossentropy

batch_size = 100

for epoch_cnt in range(2):
    
    # x_train 데이터셋의 랜덤 batch 구성하기 위해
    # 랜덤으로 섞인 idxs 배열 구성하기
    
    # 여기에 코드 작성
    idxs = np.arange(len(x_train))
    np.random.shuffle(idxs)  
    
    # 학습용 데이터셋의 batch들에 대해 학습을 진행 (앞선 실습들과 매우 유사함)
    # 주의 1: 손실함수로 softmax_crossentropy() 이용
    # 주의 2: 경사하강법을 진행할 때에는 Optimizer optim의 step() 실행
    # 주의 3: 학습 중 loss와 accuracy를 추적
    
    for batch_cnt in range(len(x_train)//batch_size):
        
        # 여기에 코드 작성
        # idxs 배열을 이용하여 batch_cnt번째 batch 구성하기
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size ]
        batch = x_train[batch_indices]  # random batch of our training data, x_train : 전체 학습 data 모음

        # prediction : batch에 대한 예측값
        prediction = model(batch) # model class의 __call__ 함수를 어떻게 호출하는가?

        # truth : batch에 대한 실제값
        truth = y_train[batch_indices] # y_train : 전체 label 모음

        # loss : softmax_crossentropy() 이용하여 loss 계산
        loss = softmax_crossentropy(prediction, truth)

        # 경사하강법 진행에 필요한 모든 편미분계수를 얻기 위해
        # loss에 대해 .backward() 메서드 실행
        loss.backward()

        # optim의 step() 실행하여 경사하강법 진행
        optim.step()

        # accuracy: 앞서 작성한 accuracy() 이용한 정확도 계산
        acc = accuracy(prediction, truth)

        # 학습 중 loss와 accuracy를 추적
        plotter.set_train_batch({"loss" : loss.item(), "accuracy" : acc}, batch_size=batch_size)
    
    # 테스트용 데이터셋의 batch들에 대해 모델을 평가
    
    for batch_cnt in range(0, len(x_test)//batch_size):
        
        # 학습 시와 마찬가지로 같은 batch size의 랜덤 batch를 구성
        
        # 여기에 코드 작성
        idxs = np.arange(len(x_test))
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]
        batch = x_test[batch_indices] 
        
        # 역전파가 필요없는 부분이므로, 그래디언트 계산이 이뤄지지 않도록
        with mg.no_autodiff:
            # prediction : 테스트 batch에 대한 예측값
            prediction = model(batch) # model class 의 __call__ 함수 사용
            
            # truth : 테스트 batch에 대한 실제값
            truth = y_test[batch_indices] # y_test : 전체 label 모음
            
            # accuracy: 앞서 작성한 accuracy() 이용한 정확도 계산
            acc = accuracy(prediction, truth)
        
        # noggin으로 test-accuracy 추가
        plotter.set_test_batch({"accuracy": acc}, batch_size=batch_size)
    
    plotter.set_train_epoch()
    plotter.set_test_epoch()
plotter.plot()

In [20]:
# 아래는 빈칸 아닙니다. img_test, label_test 앞에 다른 코드를 채우실 필요 없습니다.
_, _, img_test, label_test = load_mnist()

# MNIST의 라벨로 이루어진 튜플.
# ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
load_mnist.labels = tuple(str(i) for i in range(10))
labels = load_mnist.labels  

mnist loaded


In [21]:
def plot_model_prediction(index):
    '''index를 입력받아 index에 해당하는 이미지와 예측 결과를 시각화하는 함수'''

    true_label_index = label_test[index]
    true_label = load_mnist.labels[true_label_index]

    with mg.no_autodiff:
        # model에 입력해주는 데이터는 4차원이어야 하기 때문에
        # 축 하나가 사라지지 않도록 인덱스 설정
        prediction = model(x_test[index : index + 1])
        
        # 가장 점수가 높은 라벨이 예측값이 됨
        predicted_label_index = np.argmax(prediction.data, axis=1).item()
        predicted_label = labels[predicted_label_index]
    
    # 이미지, 실제 라벨, 예측 라벨 모두 시각화
    
    # 여기에 코드 작성
    fig, ax = plt.subplots()
    ax.imshow(img_test[index, 0], cmap="gray")
    ax.set_title(f"Predicted: {predicted_label}\nTruth: {true_label}")
    
    return fig, ax

In [22]:
# x_test의 랜덤 데이터를 시각화
index = np.random.randint(0, len(x_test))
plot_model_prediction(index)

<IPython.core.display.Javascript object>

(<Figure size 640x480 with 1 Axes>,
 <AxesSubplot:title={'center':'Predicted: 1\nTruth: 4'}>)

In [23]:
# 예측이 틀린 모든 사례 찾아서 해당 인덱스를 bad_indices에 저장
bad_indices = []

for batch_cnt in range(0, len(x_test) // batch_size):
    idxs = np.arange(len(x_test))
    batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]
    batch = x_test[batch_indices]

    with mg.no_autodiff:
        # test-batch에 대한 모델의 예측 라벨
        prediction = np.argmax(model(batch), axis=1)

        # test-batch에 대한 실제 라벨
        truth = y_test[batch_indices]
        
        # 예측과 실제가 다른 경우의 모든 인덱스를 저장
        (bad,) = np.where(prediction != truth)
        if bad.size: # 0이 아닌 경우
            bad_indices.extend(batch_indices[bad].tolist())

In [24]:
# bad_indices의 랜덤 데이터를 시각화
index = np.random.randint(0, len(bad_indices))
print(index)
plot_model_prediction(bad_indices[index])

1703


<IPython.core.display.Javascript object>

(<Figure size 640x480 with 1 Axes>,
 <AxesSubplot:title={'center':'Predicted: 1\nTruth: 2'}>)